<a href="https://colab.research.google.com/github/AvellinaLeong/NHM-Nannofossil-Segmentation-Project/blob/main/theshold_change_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Set script location to own development space
MY_DEVELOPMENT_SPACE = '/content/drive/MyDrive/development/avellina/'
import os
os.chdir(MY_DEVELOPMENT_SPACE)
!pwd
!ls

Mounted at /content/drive/
/content/drive/MyDrive/development/avellina
detectron2  Detectron2_notebooks  Mask-RCNN  Morphometrics_notebooks  output


## Change Thresholds to See if Performance Improves



*   Change Thresholds
*   Save Precision Metrics to CSV
*   Segment Test Set with different thresholds



In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 8.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
fatal: destination path 'detectron2' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.6 MB/

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.3 ; cuda:  cu121
detectron2: 0.6


In [ ]:
# Import Detectron2 and Detectron2 Logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:
import os
import torch
import copy
import numpy as np
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader, transforms as T
from detectron2.structures import BoxMode
from detectron2.data import detection_utils as utils
from detectron2.engine import DefaultTrainer
import cv2
from detectron2.utils.visualizer import Visualizer, ColorMode

## Register Datasets

In [ ]:
register_coco_instances("my_dataset_val", {}, "/content/drive/MyDrive/data/species_53/data/val/coco_val.json", "/content/drive/MyDrive/data/species_53/data/val")

## Load Model Configurations

In [ ]:
from detectron2.data import transforms as T

# Define a sequence of augmentations
augs = T.AugmentationList([
    T.RandomBrightness(0.9, 1.1),
    T.RandomFlip(prob=0.5),
    T.RandomRotation(angle=[-90, 90]),
    T.RandomSaturation(0.8, 1.2),
])

In [ ]:
# Custom data mapper
def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict["file_name"], format="BGR")

    # Get the annotations
    annos = dataset_dict.get("annotations", [])
    bbox_list = [BoxMode.convert(obj["bbox"], obj["bbox_mode"], BoxMode.XYXY_ABS) for obj in annos]
    bbox_list = np.array(bbox_list, dtype=np.float32)

    # Apply augmentations
    aug_input = T.AugInput(image, boxes=bbox_list)
    transforms = augs(aug_input)
    image = aug_input.image

    # Apply the same transforms to the annotations
    annos = [utils.transform_instance_annotations(obj, transforms, image.shape[:2]) for obj in annos]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))
    dataset_dict["instances"] = utils.filter_empty_instances(instances)

    return dataset_dict

In [ ]:
from detectron2.engine import DefaultTrainer

In [ ]:
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)

cfg = get_cfg()
cfg.OUTPUT_DIR = "/content/drive/MyDrive/data/species_53/Detectron2_Models/1"
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.VAL = ("my_dataset_val",)
cfg.DATASETS.TEST = ("my_dataset_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final_1.pth")  # Make sure this path is correct
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000   #iterations -- can change
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 # Default is 512
cfg.TEST.EVAL_PERIOD = 500  # Evaluate every 500 iterations
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Number of classes

In [ ]:
# # Set the threshold
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.51

# # Create the predictor
# predictor = DefaultPredictor(cfg)

[06/25 10:26:20 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/data/species_53/Detectron2_Models/1/model_final_1.pth ...


## Threshold Change and Save Precision Metrics to CSV

In [ ]:
import pandas as pd

In [ ]:
# Empty dataframe to store all results
results_df = pd.DataFrame()

# Define the range of thresholds to iterate over
thresholds = [i/100 for i in range(50, 91)]  # 0.51 to 0.90 in 0.01 increments

for threshold in thresholds:
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
    predictor = DefaultPredictor(cfg)

    # Evaluate the model on the validation set
    evaluator = COCOEvaluator("my_dataset_val", output_dir="./output")
    val_loader = build_detection_test_loader(cfg, "my_dataset_val")
    results = inference_on_dataset(predictor.model, val_loader, evaluator)

    # Extract relevant metrics from results
    metrics = results.get('segm')  # Try getting 'bbox' if 'segm' is not present

    # Check if metrics are available
    if metrics:
        # Add the threshold information to the metrics
        metrics['threshold'] = threshold

        # Convert metrics to a DataFrame
        metrics_df = pd.DataFrame([metrics])  # Wrap metrics in a list to create a single-row DataFrame

        # Append to the main results DataFrame
        results_df = pd.concat([results_df, metrics_df], ignore_index=True)
    else:
        print(f"No metrics available for threshold {threshold}.")

# Save the results to a CSV file
evaluation_csv_file_path = "/content/drive/MyDrive/data/species_53/Detectron2_Models/threshold_evaluation/evaluation_metrics.csv"
os.makedirs(os.path.dirname(evaluation_csv_file_path), exist_ok=True)
results_df.to_csv(evaluation_csv_file_path, index=False)

NameError: name 'pd' is not defined

## Segment images with different thresholds

In [ ]:
# Load the saved configuration from the YAML file
config_yaml_path = "/content/drive/MyDrive/data/species_53/Detectron2_Models/1/config_1.yaml"  # Path to your saved config file
cfg = get_cfg()

# Set default values for keys that might be missing in the config file
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.VAL = ("my_dataset_val",)
cfg.DATASETS.TEST = ("my_dataset_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = ""
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.TEST.EVAL_PERIOD = 500
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# Merge the configuration file with the default (debugged this way)
cfg.merge_from_file(config_yaml_path)
# Initialize the predictor with the loaded configuration
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "1", "model_final_1.pth")

In [ ]:
# Set the threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50
predictor = DefaultPredictor(cfg)

[06/28 12:23:24 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/data/species_53/Detectron2_Models/1/model_final_1.pth ...


In [ ]:
# Register the test dataset
register_coco_instances("my_dataset_test", {}, "/content/drive/MyDrive/data/species_53/data/test/coco_test.json", "/content/drive/MyDrive/data/species_53/data/test")
test_metadata = MetadataCatalog.get("my_dataset_test")
test_dataset_dicts = DatasetCatalog.get("my_dataset_test")

[06/28 12:23:37 d2.data.datasets.coco]: Loading /content/drive/MyDrive/data/species_53/data/test/coco_test.json takes 2.60 seconds.
[06/28 12:23:37 d2.data.datasets.coco]: Loaded 96 images in COCO format from /content/drive/MyDrive/data/species_53/data/test/coco_test.json


In [ ]:
# Create the output directory for the threshold
output_dir = "/content/drive/MyDrive/data/species_53/diff_threshold_segment/0.50"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Visualize and save segmentation on all test images
for d in test_dataset_dicts:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)

    # Visualize the prediction
    v = Visualizer(im[:, :, ::-1],
                   metadata=test_metadata,
                   scale=1,
                   instance_mode=ColorMode.IMAGE_BW
    )

    # Draw instance predictions with scores
    instances = outputs["instances"].to("cpu")
    boxes = instances.pred_boxes if instances.has("pred_boxes") else None
    scores = instances.scores if instances.has("scores") else None

    if boxes is not None and scores is not None:
        for box, score in zip(boxes, scores):
            box = box.cpu().numpy().astype(int)
            score = float(score)
            v.draw_box(box, edge_color="blue", line_style="solid")
            v.draw_text(
                f"{score:.2f}",
                (box[0], box[1]),
                color="red",
                font_size=2,
                horizontal_alignment="center"
            )

    out = v.draw_instance_predictions(instances)

    # Convert BGR image back to RGB
    segmented_image = out.get_image()[:, :, ::-1]

    # Construct the output file path
    file_name = os.path.basename(d["file_name"])
    output_path = os.path.join(output_dir, file_name)

    # Save the image
    cv2.imwrite(output_path, segmented_image)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/content/drive/MyDrive/development/avellina/detectron2/detectron2/layers/wrappers.py:142: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  x = F.conv2d(
